In [1]:
!pip install -q selenium==4.3.0

In [2]:
!pip install -q lxml==4.9.1

In [3]:
!pip install -q beautifulsoup4==4.11.1

In [4]:
!pip install -q mysql-connector-python==8.0.30

In [5]:
!pip install -q sqlalchemy==1.4.39

In [6]:
!pip install -q backoff==2.1.2
import backoff
backoff.__version__

'2.1.2'

In [7]:
!pip install -q pandas

In [17]:
!pip install python-dotenv

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import selenium
import time
import random
import re
import sys
# import lxml
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
from sqlalchemy.sql import text
import mysql.connector
import pandas as pd
import backoff
from datetime import datetime
import logging
from logging import handlers

In [2]:
logger = logging.getLogger('paper_main')
logger.setLevel(logging.INFO)

log_format = logging.Formatter('%(asctime)s [%(module)s] %(levelname)s [%(lineno)d] %(message)s', '%Y-%m-%d %H:%M:%S %Z')

th = handlers.TimedRotatingFileHandler(filename='app.log', when='D', backupCount=7, encoding='utf-8')
th.setFormatter(log_format)
th.setLevel(logging.INFO)
logger.addHandler(th)

ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
ch.setFormatter(log_format)
logger.addHandler(ch)

logger.info('paper_main logger ready...')

2022-08-05 17:24:25 Asia [3038962473] INFO [16] paper_main logger ready...


In [3]:
db_user = 'prints'
db_passwd = 'prints78909!'
db_name = 'kol_radar'
tbl_name = 'paper_main'
db_ip = '172.104.169.34'
engine = create_engine(f'mysql+mysqlconnector://{db_user}:{db_passwd}@{db_ip}:3306/{db_name}')
cnx = engine.connect()


In [4]:
def get_keywords_list(key_id=0):
    keyword_query = cnx.execute(f'SELECT * FROM `key_prints` where parent_key_id = {key_id}')
    keywords_dict = [dict(i) for i in keyword_query]
    # print(keywords_dict)
    return keywords_dict

In [5]:
def set_ua():
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'
    return user_agent

In [6]:
def little_sleep(min=1, max=12):
    sleep_time = random.randint(min,max)
    logger.info(f'sleep time: {sleep_time}')
    time.sleep(sleep_time)

In [7]:
@backoff.on_exception(backoff.expo,
                        Exception, 
                      max_time=10)
def chrome_init():
    chrome_opt = webdriver.ChromeOptions()
    chrome_opt.add_argument('--headless')
    chrome_opt.add_argument('--no-sandbox')
    chrome_opt.add_argument('--ignore-ssl-errors=yes')
    chrome_opt.add_argument('--ignore-certificate-errors')
    chrome_opt.add_argument(f'user-agent={set_ua()}')
    chrome_opt.add_argument("--incognito")  # 使用無痕模式。用 selenium開瀏覽器已經很乾淨了，但疑心病重的可以用一下
    try:
        driver = webdriver.Remote(
            command_executor='http://selenium-hub:4444/wd/hub',
            options=chrome_opt
        )
    except Exception as e:
        logger.error(f'chrome init error: {e}')
        raise Exception(e)
    return driver

In [8]:
def crawl_home_page(url, driver):
    driver.get(url)
    title = driver.title
    logger.info(title)
    driver.set_window_size(1920, 1080)
    driver.get_screenshot_as_file("論文全文資料庫.png")
    ccd = ''
    if len(driver.current_url) > 0:
        try:
            ccd = re.findall(r'ccd=(.*?)/', driver.current_url)[0]
        except:
            logger.error(f'driver.current_url: {driver.current_url}')
    little_sleep(1,3)
    return ccd, driver


In [9]:
def go_keyword_search_page(driver):
    try:
        # 正常
        driver.find_element(By.XPATH, '//a[@title="指令查詢"]').click()
        driver.get_screenshot_as_file("指令查詢.png")
        little_sleep(1,3)
    except:
        driver.get_screenshot_as_file("keyword_search_page_ERROR.png")
        img_element = driver.find_element(By.XPATH, '/html/body/div/form/div[1]/img')
        logger.error(f'ERR: {img_element}')
    return driver


In [10]:
my_keys = '"汽車" or "家庭與社區" or "房地產" or "工商業" or "美妝與個人護理" or "場所與禮品" or "網際網路與電信" or "家居與園藝" or "食品與雜貨" or "保健" or "財經" or "藝術與娛樂" or "運動與健身" or "興趣與休閒" or "工作與教育" or "旅遊" or "法律與政府" or "電腦與消費性電子產品" or "餐飲與夜生活" or "服飾"'


In [11]:
def keyword_search_result(keywords, driver):
    search_result_count_xpath = '//*[@id="bodyid"]/form/div/table/tbody/tr[1]/td[2]/table/tbody/tr[4]/td/div[1]/table/tbody/tr[2]/td/table[2]/tbody/tr[2]/td[2]/span[2]'
    driver.find_element(By.XPATH, '//*[@id="ysearchinput0"]').send_keys(my_keys)
    driver.find_element(By.XPATH, '//*[@id="gs32search"]').click()
    search_result_count_element = WebDriverWait(driver, timeout=120).until(lambda d: d.find_element(By.XPATH, search_result_count_xpath))
    driver.get_screenshot_as_file("指令查詢_result.png")
    total_count = 0
    if search_result_count_element is not None:
        total_count = search_result_count_element.text
    # ccd = re.findall(r'ccd=(.*?)/', driver.current_url)[0]
    little_sleep(1,3)
    return total_count, driver


In [12]:
def parse_paper_html(page_source):
    # soup = BeautifulSoup(driver.page_source)
    soup = BeautifulSoup(page_source, features="lxml")
    page_static_url=student=student_en=paper_title=paper_title_en=advisor=advisor_en=exam_committee=exam_committee_en=exam_date=degree_category=school=department=phylum=paper_class=paper_type=pub_year=graduaction_year=lang=paper_page=keywords_tw=keywords_en=abstract_note=abstract_note_en=directory=references=paper_paper=qrcode= None

    # 論文基本資料
    tbody = soup.find('table',{'id':'format0_disparea'})
    if tbody is None:
        driver.get_screenshot_as_file('tbody_none.png')
        raise Exception(f'fetch tbody is None, page source size: {len(page_source)}')

    # 連結網址
    page_static_url = tbody.find('input',{'id':'fe_text1'})['value']
    logger.info(f'連結網址： {page_static_url}')

    # 研究生
    for element in tbody.select('tr'):
        if '研究生' in element.text:
            student = element.find('a').text
            break    
    # print('研究生:', student)

    # 研究生_外文
    for element in tbody.select('tr'):
        if '研究生(外文)' in element.text:
            student_en = element.find('a').text
            break    
    # print('研究生_外文:', student)

    # 論文名稱
    for element in tbody.select('tr'):
        if '論文名稱' in element.text:
            paper_title = element.find('td').text
            break    
    # print('論文名稱:', paper_title)

    # 論文名稱_外文
    for element in tbody.select('tr'):
        if '論文名稱(外文)' in element.text:
            paper_title_en = element.find('td').text
            break   
    # print('論文名稱_外文:', paper_title_en)

    # 指導教授
    for element in tbody.select('tr'):
        if '指導教授' in element.text:
            advisor = element.find('td').text
            break   
    # print('指導教授:', advisor)

    # 指導教授_外文
    for element in tbody.select('tr'):
        if '指導教授(外文)' in element.text:
            advisor_en = element.find('td').text
            break   
    # print('指導教授_外文:', advisor_en)

    # 學位類別
    for element in tbody.select('tr'):
        if '學位類別' in element.text:
            degree_category = element.find('td').text
            break   
    # print('學位類別:', degree_category)

    # 校院名稱
    for element in tbody.select('tr'):
        if '校院名稱' in element.text:
            school = element.find('td').text
            break   
    # print('校院名稱:', school)

    # 系所名稱
    for element in tbody.select('tr'):
        if '系所名稱' in element.text:
            department = element.find('td').text
            break   
    # print('系所名稱:', department)

    # 學門
    for element in tbody.select('tr'):
        if '學門' in element.text:
            phylum = element.find('td').text
            break   
    # print('學門:', phylum)

    # 學類
    for element in tbody.select('tr'):
        if '學類' in element.text:
            paper_class = element.find('td').text
            break   
    # print('學類:', paper_class)

    # 論文出版年
    for element in tbody.select('tr'):
        if '論文出版年' in element.text:
            pub_year = element.find('td').text
            break   
    # print('論文出版年:', pub_year)

    # 畢業學年度
    for element in tbody.select('tr'):
        if '畢業學年度' in element.text:
            pub_year = element.find('td').text
            break   
    # print('畢業學年度:', pub_year)

    # 語文別
    for element in tbody.select('tr'):
        if '語文別' in element.text:
            lang = element.find('td').text
            break   
    # print('語文別:', lang)

    # 論文頁數
    for element in tbody.select('tr'):
        if '論文頁數' in element.text:
            paper_page = element.find('td').text
            if paper_page == '':
                paper_page = None
            if paper_page is not None:
                paper_page = int(paper_page)
            break   
    # print('論文頁數:', paper_page)

    # 中文關鍵詞
    for element in tbody.select('tr'):
        if '中文關鍵詞' in element.text:
            keywords_tw = element.find('td').text
            break   
    # print('中文關鍵詞:', keywords_tw)

    # 外文關鍵詞
    for element in tbody.select('tr'):
        if '外文關鍵詞' in element.text:
            keywords_en = element.find('td').text
            break   
    # print('外文關鍵詞:', keywords_en)

    # 被引用
    for element in tbody.select('tr'):
        if '相關次數' in element.text:
            refed = element.findAll('li')[0].text
            refed = re.sub('被引用:','',refed)
            break   
    # print('被引用:', refed)

    # 點閱
    for element in tbody.select('tr'):
        if '相關次數' in element.text:
            click = element.findAll('li')[1].text
            break   
    # print('點閱:', click)

    # 下載
    for element in tbody.select('tr'):
        if '相關次數' in element.text:
            download_times = element.findAll('li')[3].text
            download_times = re.sub('下載:','',download_times)
            break   
    # print('下載:', download_times)

    # 書目收藏
    for element in tbody.select('tr'):
        if '相關次數' in element.text:
            collection = element.findAll('li')[4].text
            collection = re.sub('書目收藏:','',collection)
            break   
    # print('書目收藏:', collection)

    stdncl2 = soup.find_all('td',{'class':'stdncl2'})
    # print(f'stdncl2 size: [{len(stdncl2)}]')
    # for each_info in stdncl2:
    #     print(f'info: {each_info.text}')
    # print(f'摘要: {stdncl2[0].text}')
    # print(f'摘要_en: {stdncl2[1].text}')
    # print(f'目次: {stdncl2[2].text}')
    # print(f'參考文獻: {stdncl2[3].text}')

    # 摘要
    try:
        # abstract_note = soup.find('td',{'class':'stdncl2'}).text
        abstract_note = stdncl2[0].text if len(stdncl2) > 1 and stdncl2[0] is not None else ''
    except:
        abstract_note = ''
    # print('摘要：', abstract_note)

    # 摘要_en
    try:
        # abstract_note_en = soup.find('td',{'class':'stdncl2'}).text
        abstract_note_en = stdncl2[1].text if len(stdncl2) >= 2 and stdncl2[1] is not None else ''
    except:
        abstract_note_en = ''
    # print(f'摘要_en：{abstract_note_en}')

    # 目次
    try:
        directory = stdncl2[2].text if len(stdncl2) >= 3 and stdncl2[2] is not None else ''
    except:
        directory = ''

    # 參考文獻
    try:
        references = stdncl2[3].text if len(stdncl2) >= 4 and stdncl2[3] is not None else ''
    except:
        references = ''

    # 口試委員
    for element in tbody.select('tr'):
        if '口試委員' in element.text:
            exam_committee = element.find('td').text if element.find('td') is not None else ''
            break
        # print('口試委員:', exam_committee)

    # 口試委員_外文
    exam_committee_en = ''
    for element in tbody.select('tr'):
        if '口試委員(外文)' in element.text:
            exam_committee_en = element.find('td').text
            break   
        # print('口試委員_外文:', exam_committee_en)

    # 口試日期
    for element in tbody.select('tr'):
        if '口試日期' in element.text:
            exam_date = element.find('td').text
            print(f'fetch exam date: {exam_date}, type: {type(exam_date)}')
            if len(exam_date) == 0:
                exam_date = None
            if exam_date is not None:
                exam_date = datetime.strptime(exam_date, '%Y-%m-%d')
            break
        if exam_date is not None:
            logger.info(f'口試日期: {exam_date}, type: {type(exam_date)}')

    #論文種類
    for elememt in tbody.select('tr'):
        if '論文種類' in element.text:
            paper_type = element.find('td').text
            break
        # print(f'論文種類: {paper_type}')

    # 引用
    refer_html = str(soup.find('div',{'style':'padding:10px;text-align:left;'}))
    
    paper = {
        'paper_url': page_static_url,
        'author': student,
        'author_en': student_en,
        'title': paper_title,
        'title_en': paper_title_en,
        'advisor': advisor,
        'advisor_en': advisor_en,
        'interviewer': exam_committee,
        'interviewer_en': exam_committee_en,
        'exam_date': exam_date,
        'paper_type': degree_category,
        'school': school,
        'department_lv1': department,
        'department_lv2': phylum,
        'department_lv3': paper_class,
        'paper_type': paper_type,
        'publish_year': pub_year,
        'graduaction_year': graduaction_year, #畢業學年度
        'lang': lang, #語文別
        'paper_page': paper_page, #論文頁數
        'paper_keyword_cn': keywords_tw,
        'paper_keyword_en': keywords_en,
        'paper_summary': abstract_note,
        'paper_summary_en': abstract_note_en, #外文摘要
        'directory': directory, #目次
        # '紙本論文': paper_paper,
        # 'QRCode': qrcode,
        'quote': references,
        'refer_html': refer_html,
        'page_source': page_source
    }
    return paper


In [13]:
@backoff.on_exception(backoff.expo,
                      (selenium.common.exceptions.UnexpectedAlertPresentException, 
                        Exception, 
                        mysql.connector.errors.OperationalError),
                      max_time=10)
def scrape_each_paper(search_result_count, ccd, driver, start_num=1):
    curr_page_num = 0
    search_result_count = int(search_result_count)
    if search_result_count > 0:
        # paper_df = pd.DataFrame()
        insert_sql = text("""INSERT INTO `paper_main` (`paper_url`, `author`, `author_en`, `title`, `title_en`, `advisor`, `advisor_en`, `interviewer`, `interviewer_en`, `exam_date`, `paper_type`, `school`, `department_lv1`, `department_lv2`, `department_lv3`, `publish_year`, `graduaction_year`, `lang`, `paper_page`, `paper_keyword_cn`, `paper_keyword_en`, `paper_summary`, `paper_summary_en`, `directory`, `quote`, `refer_html`, `page_source`) VALUES
(:paper_url, :author, :author_en, :title, :title_en, :advisor, :advisor_en, :interviewer, :interviewer_en, :exam_date, :paper_type, :school, :department_lv1, :department_lv2, :department_lv3, :publish_year, :graduaction_year, :lang, :paper_page, :paper_keyword_cn, :paper_keyword_en, :paper_summary, :paper_summary_en, :directory, :quote, :refer_html, :page_source);
""")
        update_sql = text("""
            update `paper_main`
            set 
            `author` = :author,
            `author_en` = :author_en,
            `title` = :title,
            `title_en` = :title_en,
            `advisor` = :advisor,
            `advisor_en` = :advisor_en,
            `interviewer` = :interviewer,
            `interviewer_en` = :interviewer_en,
            `exam_date` = :exam_date,
            `paper_type` = :paper_type,
            `school` = :school,
            `department_lv1` = :department_lv1,
            `department_lv2` = :department_lv2,
            `department_lv3` = :department_lv3,
            `publish_year` = :publish_year,
            `graduaction_year` = :graduaction_year,
            `lang` = :lang,
            `paper_page` = :paper_page,
            `paper_keyword_cn` = :paper_keyword_cn,
            `paper_keyword_en` = :paper_keyword_en,
            `paper_summary` = :paper_summary,
            `paper_summary_en` = :paper_summary_en,
            `directory` = :directory,
            `quote` = :quote,
            `refer_html` = :refer_html,
            `page_source` = :page_source
            where `paper_url` = :paper_url
        """)
        count_sql = text("""select count(*) as cnt from `paper_main` where `paper_url` = :paper_url""")
        i = start_num
        for i in range(1, search_result_count):
            if i % 200 == 0:
                ccd, driver = crawl_home_page(url, driver)
                print(f'new ccd: {ccd}, curr url: {driver.current_url}')
                driver = keyword_search_page(driver)
                total_count, driver = keyword_search_result(my_keys, driver)
                little_sleep(1,3)
            logger.info(f'ccd: {ccd}')
            if ccd is None:
                ccd, driver = crawl_home_page(url, driver)
            page_url = f'https://ndltd.ncl.edu.tw/cgi-bin/gs32/gsweb.cgi/ccd={ccd}/record?r1={i}&h1=0'
            logger.info(f'page url: {page_url}')
            try:
                driver.get(page_url)
                paper = parse_paper_html(driver.page_source)
                # paper_df = paper_df.append(paper, ignore_index=True)
                # paper_df = pd.DataFrame(paper)
                count_result = cnx.execute(count_sql, **paper)
                count_dict = [dict(i) for i in count_result]
                cnt = int(count_dict[0]['cnt'])
                logger.info(f'[{i}]page url: {page_url}, count: {cnt}')
                if cnt == 0:
                    cnx.execute(insert_sql, **paper)
                else:
                    cnx.execute(update_sql, **paper)
            except Exception as e:
                logger.error(f'[{i}]page: {page_url} error')
                logger.error(e)
                driver.get_screenshot_as_file(f"page_{driver.title}_error.png")
                # paper_df.to_csv(f'error/{page_url}_error.csv')
            finally:
                curr_page_num = i
                little_sleep(3,12)
            logger.info('='*11)
    return curr_page_num, driver


In [14]:
def crawl_paper_pages(keywords, curr_page_num, driver):
    try:
        driver = go_keyword_search_page(driver)
        total_count, driver = keyword_search_result(keywords, driver)
        logger.info(f'結果筆數: {total_count}, ccd: {ccd}')
        curr_page_num, driver = scrape_each_paper(total_count, ccd, driver, curr_page_num)
    except Exception as e:
        logger.error(e)


In [15]:
url = 'https://ndltd.ncl.edu.tw/'
logger.info(f'url: {url}')
# driver = None
search_paper_result = {}
curr_page_num = 1
total_count = 0


2022-08-05 17:24:32 Asia [985876430] INFO [2] url: https://ndltd.ncl.edu.tw/


In [16]:
try:
    driver = chrome_init()
    ccd, driver = crawl_home_page(url, driver)
    logger.info(driver.current_url)
    while True:
        try:
            driver = go_keyword_search_page(driver)
            total_count, driver = keyword_search_result(my_keys, driver)
            logger.info(f'結果筆數: {total_count}, ccd: {ccd}')
            curr_page_num, driver = scrape_each_paper(total_count, ccd, driver, curr_page_num)
            search_paper_result = {
                'key_words': my_keys,
                'total_count': total_count,
                'curr_page_num' : curr_page_num
            }
            if curr_page_num >= total_count:
                break;
        except Exception as e:
            logger.error(f'curr page num:{curr_page_num}, total count: {total_count}, keywords: {my_keys}')
            logger.error(e)


except Exception as e:
    # search_paper_result = {
    #     'key_words': my_keys,
    #     'total_count': total_count,
    #     'curr_page_num' : curr_page_num
    # }
    logger.error(e)
    # logger.error(f'search paper result: {search_paper_result}')
finally:
    if driver is not None:
        driver.close()
        driver.quit()
        logger.info(f'driver: {driver}')

2022-08-05 17:24:41 Asia [3831567472] INFO [4] 臺灣博碩士論文知識加值系統：自由的博碩士學位論文全文資料庫 (現在位置：首頁)
2022-08-05 17:24:49 Asia [1540640465] INFO [3] sleep time: 3
2022-08-05 17:24:52 Asia [3014959867] INFO [4] https://ndltd.ncl.edu.tw/cgi-bin/gs32/gsweb.cgi/ccd=wGo8a0/login?jstimes=1&loadingjs=1&o=dwebmge&ssoauth=1&cache=1659720276928
2022-08-05 17:24:53 Asia [1540640465] INFO [3] sleep time: 3
2022-08-05 17:25:01 Asia [1540640465] INFO [3] sleep time: 2
2022-08-05 17:25:03 Asia [3014959867] INFO [9] 結果筆數:  160583 , ccd: wGo8a0
2022-08-05 17:25:03 Asia [665092158] INFO [54] ccd: wGo8a0
2022-08-05 17:25:03 Asia [665092158] INFO [58] page url: https://ndltd.ncl.edu.tw/cgi-bin/gs32/gsweb.cgi/ccd=wGo8a0/record?r1=1&h1=0
2022-08-05 17:25:04 Asia [1296822138] INFO [14] 連結網址： https://hdl.handle.net/11296/pm2by2
2022-08-05 17:25:04 Asia [665092158] INFO [67] [1]page url: https://ndltd.ncl.edu.tw/cgi-bin/gs32/gsweb.cgi/ccd=wGo8a0/record?r1=1&h1=0, count: 1
2022-08-05 17:25:05 Asia [1540640465] INFO [3] sleep 

KeyboardInterrupt: 